In [1]:
import numpy as np
import pandas as pd
import snscrape.modules.twitter as sntwitter
import itertools
from datetime import datetime, timedelta
from cleanco import basename

In [15]:
elec_df = pd.read_excel('nlrb_elections_cleaned.xlsx')


In [16]:
#Clean company names for easier comparison
elec_df['Cleaned Case Name'] = elec_df['Case Name'].apply(lambda x: basename((x.split('A/K/A')[-1]).split('D/B/A')[-1]))
elec_df.head(10)

,Region,Case Number,Case Name,Status,Date Filed,Date Closed,Reason Closed,City,States & Territories,Unit ID,...,Voting Unit (Unit B),Voting Unit (Unit C),Voting Unit (Unit D),Participation Ratio,Location,Location String,Geo Location,Latitude,Longitude,Cleaned Case Name
0,"Region 01, Boston, Massachusetts",01-RC-090869,Brattleboro Food Co-op,Closed,10/9/2012,11/26/2012,Certific. of Representative,Brattleboro,VT,A,...,NaN,NaN,NaN,0.838028,"Brattleboro, VT, US","Brattleboro, VT, US","Brattleboro, Vermont, United States",42.85092,-72.55787,Brattleboro Food
1,"Region 10, Atlanta, Georgia",10-RC-090329,"JOHNSON CONTROLS, INC.",Closed,10/1/2012,11/26/2012,Certific. of Representative,COTTONDALE,AL,A,...,NaN,NaN,NaN,0.970297,"COTTONDALE, AL, US","COTTONDALE, AL, US","Cottondale, Alabama, United States",33.18956,-87.45167,JOHNSON CONTROLS
2,"Region 18, Minneapolis, Minnesota",18-RC-090794,"A.S.V., INC. A/K/A TEREX",Closed,10/5/2012,11/21/2012,Certification of Results,Grand Rapids,MN,A,...,NaN,NaN,NaN,0.982759,"Grand Rapids, MN, US","Grand Rapids, MN, US","Grand Rapids, Minnesota, United States",47.23717,-93.53021,TEREX
3,"Region 29, Brooklyn, New York",29-RD-091658,"Xerox State and Local Solutions, Inc.",Closed,10/19/2012,11/27/2012,Certific. of Representative,Staten Island,NY,A,...,NaN,NaN,NaN,0.870079,"Staten Island, NY, US","Staten Island, NY, US","Staten Island, New York, United States",40.56233,-74.13986,Xerox State and Local Solutions
4,"Region 09, Cincinnati, Ohio",09-RC-090819,"AWP, INC D/B/A AREA WIDE PROTECTIVE",Closed,10/5/2012,11/26/2012,Certification of Results,CINCINNATI,OH,A,...,NaN,NaN,NaN,0.703125,"CINCINNATI, OH, US","CINCINNATI, OH, US","Cincinnati, Ohio, United States",39.12711,-84.51439,AREA WIDE PROTECTIVE
5,"Region 21, Los Angeles, California",21-RC-089564,"FIRST STUDENT, INC.",Closed,9/19/2012,11/26/2012,Certification of Results,GARDENA,CA,A,...,NaN,NaN,NaN,0.742268,"GARDENA, CA, US","GARDENA, CA, US","Gardena, California, United States",33.88835,-118.30896,FIRST STUDENT
6,"Region 32, Oakland, California",32-RC-090886,"Southern Monterey County Memorial Hospital, In...",Closed,10/9/2012,5/7/2013,Certification of Results,King City,CA,A,...,NaN,NaN,NaN,0.897887,"King City, CA, US","King City, CA, US","King City, California, United States",36.21274,-121.12603,"Southern Monterey County Memorial Hospital, In..."
7,"Region 28, Phoenix, Arizona",28-RC-092433,"Bellagio, LLC",Closed,11/1/2012,11/28/2012,Certific. of Representative,Las Vegas,NV,A,...,NaN,NaN,NaN,0.929032,"Las Vegas, NV, US","Las Vegas, NV, US","Las Vegas, Nevada, United States",36.17497,-115.13722,Bellagio
8,"Region 08, Cleveland, Ohio",08-RC-088385,"Horseshoe Cleveland Mgt., LLC/Caesars Entertai...",Closed,8/30/2012,11/29/2012,Certific. of Representative,Cleveland,OH,A,...,NaN,NaN,NaN,0.718182,"Cleveland, OH, US","Cleveland, OH, US","Cleveland, Ohio, United States",41.49950,-81.69541,"Horseshoe Cleveland Mgt., LLC/Caesars Entertai..."
9,"Region 21, Los Angeles, California",21-RC-092165,"CHAPMAN MEDICAL CENTER, INC.",Closed,10/29/2012,6/21/2013,Certification of Results,ORANGE,CA,A,...,NaN,NaN,NaN,0.673171,"ORANGE, CA, US","ORANGE, CA, US","Orange, California, United States",33.78779,-117.85311,CHAPMAN MEDICAL CENTER


In [17]:
elec_df['Tweets - General'] = None
elec_df['Tweets - Union'] = None
elec_df['Tweets - Labor Org'] = None
elec_df['Tweets - Case Name'] = None


for idx in elec_df.index:
    print("Scraping tweets for case: ", elec_df['Case Name'][idx])
    lat_long = str(elec_df['Latitude'][idx]) + ", " + str(elec_df['Longitude'][idx]) + ", 50km"
    tally_date = datetime.strptime(str(elec_df['Tally Date'][idx]), '%m/%d/%Y')
    start_date = tally_date - timedelta(days=45)    #days prior
    
    general_twt_df = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper('geocode:"{}" since:"{}" until:"{}"'.format(lat_long, start_date, tally_date)).get_items(), 1000))
    if general_twt_df.empty:
        continue
    else:
        elec_df['Tweets - General'][idx] = list(general_twt_df['renderedContent'])


    union_twt_df = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper('union geocode:"{}" since:"{}" until:"{}"'.format(lat_long, start_date, tally_date)).get_items(), 1000))
    if union_twt_df.empty:
        pass
    else:
        elec_df['Tweets - Union'][idx] = list(union_twt_df['renderedContent'])

    labor_org = str(elec_df['Labor Union1'][idx])
    labor_twt_df = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper('{} geocode:"{}" since:"{}" until:"{}"'.format(labor_org, lat_long, start_date, tally_date)).get_items(), 1000))
    if labor_twt_df.empty:
        pass
    else:
        elec_df['Tweets - Labor Org'][idx] = list(labor_twt_df['renderedContent'])

    case_name = str(elec_df['Cleaned Case Name'][idx])
    case_twt_df = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper('{} geocode:"{}" since:"{}" until:"{}"'.format(case_name, lat_long, start_date, tally_date)).get_items(), 1000))
    if case_twt_df.empty:
        pass
    else:
        elec_df['Tweets - Case Name'][idx] = list(case_twt_df['renderedContent'])

Scraping tweets for case:  Brattleboro  Food Co-op


C:\Users\rowen\AppData\Local\Temp\ipykernel_17964\2750236657.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elec_df['Tweets - General'][idx] = list(general_twt_df['renderedContent'])
C:\Users\rowen\AppData\Local\Temp\ipykernel_17964\2750236657.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elec_df['Tweets - Union'][idx] = list(union_twt_df['renderedContent'])
C:\Users\rowen\AppData\Local\Temp\ipykernel_17964\2750236657.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

Scraping tweets for case:  JOHNSON CONTROLS, INC.


C:\Users\rowen\AppData\Local\Temp\ipykernel_17964\2750236657.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elec_df['Tweets - General'][idx] = list(general_twt_df['renderedContent'])
C:\Users\rowen\AppData\Local\Temp\ipykernel_17964\2750236657.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elec_df['Tweets - Union'][idx] = list(union_twt_df['renderedContent'])


Scraping tweets for case:  A.S.V., INC. A/K/A TEREX


...

C:\Users\rowen\AppData\Local\Temp\ipykernel_17964\2750236657.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elec_df['Tweets - General'][idx] = list(general_twt_df['renderedContent'])
C:\Users\rowen\AppData\Local\Temp\ipykernel_17964\2750236657.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elec_df['Tweets - Union'][idx] = list(union_twt_df['renderedContent'])


Scraping tweets for case:  Mayo Clinic Health System - Austin


C:\Users\rowen\AppData\Local\Temp\ipykernel_17964\2750236657.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elec_df['Tweets - General'][idx] = list(general_twt_df['renderedContent'])
C:\Users\rowen\AppData\Local\Temp\ipykernel_17964\2750236657.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elec_df['Tweets - Union'][idx] = list(union_twt_df['renderedContent'])


Scraping tweets for case:  Allied Waste Transportation, Inc. d/b/a Allied Waste Services of Phoenix/Republic Services of Phoenix/Paradise Transfer Station


C:\Users\rowen\AppData\Local\Temp\ipykernel_17964\2750236657.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elec_df['Tweets - General'][idx] = list(general_twt_df['renderedContent'])
C:\Users\rowen\AppData\Local\Temp\ipykernel_17964\2750236657.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elec_df['Tweets - Union'][idx] = list(union_twt_df['renderedContent'])


In [18]:
elec_df.head(20)

,Region,Case Number,Case Name,Status,Date Filed,Date Closed,Reason Closed,City,States & Territories,Unit ID,...,Location,Location String,Geo Location,Latitude,Longitude,Cleaned Case Name,Tweets - General,Tweets - Union,Tweets - Labor Org,Tweets - Case Name
0,"Region 01, Boston, Massachusetts",01-RC-090869,Brattleboro Food Co-op,Closed,10/9/2012,11/26/2012,Certific. of Representative,Brattleboro,VT,A,...,"Brattleboro, VT, US","Brattleboro, VT, US","Brattleboro, Vermont, United States",42.85092,-72.55787,Brattleboro Food,"[Harry Potter week yesssss!!!!!, Lmao when my ...",[@notch Preprocess all the meshes into a union...,None,"[Volunteers, food needed for Thanksgiving - bi..."
1,"Region 10, Atlanta, Georgia",10-RC-090329,"JOHNSON CONTROLS, INC.",Closed,10/1/2012,11/26/2012,Certific. of Representative,COTTONDALE,AL,A,...,"COTTONDALE, AL, US","COTTONDALE, AL, US","Cottondale, Alabama, United States",33.18956,-87.45167,JOHNSON CONTROLS,"[@aj_mada I'm here, ""Are pigs flying in Tuscal...","[I'm at eCO Credit Union (Alabaster, AL) 4sq.c...",None,None
2,"Region 18, Minneapolis, Minnesota",18-RC-090794,"A.S.V., INC. A/K/A TEREX",Closed,10/5/2012,11/21/2012,Certification of Results,Grand Rapids,MN,A,...,"Grand Rapids, MN, US","Grand Rapids, MN, US","Grand Rapids, Minnesota, United States",47.23717,-93.53021,TEREX,[@SHAdEDAddY geeze if there were only one of y...,None,None,None
3,"Region 29, Brooklyn, New York",29-RD-091658,"Xerox State and Local Solutions, Inc.",Closed,10/19/2012,11/27/2012,Certific. of Representative,Staten Island,NY,A,...,"Staten Island, NY, US","Staten Island, NY, US","Staten Island, New York, United States",40.56233,-74.13986,Xerox State and Local Solutions,"[- Vallase de ahy !, I'm at Journal Square PAT...","[I'm at Grand Union Hotel (New York, NY) 4sq.c...",None,None
4,"Region 09, Cincinnati, Ohio",09-RC-090819,"AWP, INC D/B/A AREA WIDE PROTECTIVE",Closed,10/5/2012,11/26/2012,Certification of Results,CINCINNATI,OH,A,...,"CINCINNATI, OH, US","CINCINNATI, OH, US","Cincinnati, Ohio, United States",39.12711,-84.51439,AREA WIDE PROTECTIVE,"[Love &amp; Happiness .., #np Al Green .., let...",[@t_marsh83 @frontmantrue @fredthompson agreed...,None,None
5,"Region 21, Los Angeles, California",21-RC-089564,"FIRST STUDENT, INC.",Closed,9/19/2012,11/26/2012,Certification of Results,GARDENA,CA,A,...,"GARDENA, CA, US","GARDENA, CA, US","Gardena, California, United States",33.88835,-118.30896,FIRST STUDENT,[@SantanaClaus96 Pay attention if I have to y...,[mi banda favorita? mmmm\nteenangels(o como se...,None,None
6,"Region 32, Oakland, California",32-RC-090886,"Southern Monterey County Memorial Hospital, In...",Closed,10/9/2012,5/7/2013,Certification of Results,King City,CA,A,...,"King City, CA, US","King City, CA, US","King City, California, United States",36.21274,-121.12603,"Southern Monterey County Memorial Hospital, In...",[Two weeks ago Iwent to go see you youwere get...,[The sky looks so pretty on the way to our com...,None,None
7,"Region 28, Phoenix, Arizona",28-RC-092433,"Bellagio, LLC",Closed,11/1/2012,11/28/2012,Certific. of Representative,Las Vegas,NV,A,...,"Las Vegas, NV, US","Las Vegas, NV, US","Las Vegas, Nevada, United States",36.17497,-115.13722,Bellagio,"[@RockinRobin430 Huh. What plane was he in?, L...","[I'm at UNLV Student Union (Las Vegas, NV) 4sq...",None,[Antes de salir a los Latin Grammy! Con todo p...
8,"Region 08, Cleveland, Ohio",08-RC-088385,"Horseshoe Cleveland Mgt., LLC/Caesars Entertai...",Closed,8/30/2012,11/29/2012,Certific. of Representative,Cleveland,OH,A,...,"Cleveland, OH, US","Cleveland, OH, US","Cleveland, Ohio, United States",41.49950,-81.69541,"Horseshoe Cleveland Mgt., LLC/Caesars Entertai...","[""@ekennerly_25: @newborn_crotch @ned_flander5...","[I'm at Student Union (Akron, Ohio) w/ 2 other...",None,None
9,"Region 21, Los Angeles, California",21-RC-092165,"CHAPMAN MEDICAL CENTER, INC.",Closed,10/29/2012,6/21/2013,Certification of Results,ORANGE,CA,A,...,"ORANGE, CA, US","ORANGE, CA, US"

In [19]:
elec_df.to_excel("elections_with_tweets.xlsx")